# Evolutionary algorithm playground
Edit the parameters below and run the notebook from top to bottom to experiment with the EA.


## Imports and setup


In [ ]:
import random
from pathlib import Path
import sys

PROJECT_ROOT = None
for candidate in [Path.cwd().resolve(), *Path.cwd().resolve().parents]:
    module_root = candidate / "backend" / "src"
    if module_root.exists():
        PROJECT_ROOT = candidate
        if str(module_root) not in sys.path:
            sys.path.insert(0, str(module_root))
        break

if PROJECT_ROOT is None:
    raise RuntimeError("Could not locate backend/src. Run the notebook from the repo or adjust the path setup.")

from ver0.evolver import EAConfig, evolve, init_population, make_next_generation, make_random_layout, mutate, evaluate_population
from ver0.fitness import Weights
from ver0.grid_encoder import encode_floorplan_to_grid
from ver0.vars import (
    DEFAULT_GRID_SIZE,
    POPULATION_SIZE,
    GENERATIONS,
    CROSSOVER_RATE,
    MUTATION_RATE,
    TOURNAMENT_K,
    ELITE_FRACTION,
    RANDOM_SEED,
    QUADRANT_WEIGHT,
    OVERLAP_WEIGHT,
    AREA_WEIGHT,
    COMPACTNESS_WEIGHT,
    ADJACENCY_WEIGHT,
)


## Parameters


In [ ]:
# Not initilising FLOOR_ID = random.randint(1, 970) - initialised down below
GRID_SIZE = DEFAULT_GRID_SIZE

POPULATION_SIZE = 32
GENERATIONS = 512
CROSSOVER_RATE = 0.6
MUTATION_RATE = 0.4
TOURNAMENT_K = 4
ELITE_FRACTION = 0.05
RANDOM_SEED = random.randint(0, 1_000_000)

QUADRANT_WEIGHT = 1.0
OVERLAP_WEIGHT = 3.0
AREA_WEIGHT = 1.0
COMPACTNESS_WEIGHT = 0.5
ADJACENCY_WEIGHT = 0.5

EA_CONFIG = EAConfig(
    population_size=POPULATION_SIZE,
    generations=GENERATIONS,
    crossover_rate=CROSSOVER_RATE,
    mutation_rate=MUTATION_RATE,
    tournament_k=TOURNAMENT_K,
    elite_fraction=ELITE_FRACTION,
    random_seed=RANDOM_SEED,
    weights=Weights(
        quadrant=QUADRANT_WEIGHT,
        overlap=OVERLAP_WEIGHT,
        area=AREA_WEIGHT,
        compactness=COMPACTNESS_WEIGHT,
        adjacency=ADJACENCY_WEIGHT,
    ),
)


## Load floor description sample

In [ ]:
FLOOR_ID = random.randint(1, 970)

import json
floor_dir = PROJECT_ROOT / "backend" / "data" / "processed" / "floor_plans" / f"floor{FLOOR_ID:03d}"
meta_path = floor_dir / "metadata.json"
support_text = None
if meta_path.exists():
    meta = json.loads(meta_path.read_text())
    support_text = meta.get("supporting_text") or meta.get("support_text") or meta.get("text") or meta.get("scene_description")
else:
    meta = None
sample = encode_floorplan_to_grid(floor_dir, grid_size=GRID_SIZE)
print("GridSample summary")
print("------------------")
print(f"floor_id:      {FLOOR_ID}")
print(f"grid_size:     {sample.grid_size}")
print(f"num rooms:     {len(sample.rooms)}")
print()
print("EA input text (if available):")
print(support_text if support_text else "[no support text found]")
print()


## Run the evolutionary algorithm


In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
SNAPSHOT_COUNTER = max(1, (EA_CONFIG.generations // 10))
history = []
snapshots = []
counter = 0
rng = random.Random(EA_CONFIG.random_seed)
population = init_population(sample, EA_CONFIG, rng, make_random_layout)
def layout_to_grid(sample, genome):
    grid_size = sample.grid_size
    grid = -1 * np.ones((grid_size, grid_size), dtype=int)
    for idx, spec in enumerate(sample.rooms):
        cells = genome.layout.placement.get(spec.name, [])
        for r, c in cells:
            if 0 <= r < grid_size and 0 <= c < grid_size:
                grid[r, c] = idx
    return grid
for gen in range(EA_CONFIG.generations):
    if gen > 0:
        population = make_next_generation(sample, population, EA_CONFIG, rng, make_random_layout, mutate)
    evaluate_population(sample, population, EA_CONFIG)
    best_genome = min(population, key=lambda g: g.fitness if g.fitness is not None else float('inf'))
    history.append(best_genome.fitness)
    counter += 1
    if counter == SNAPSHOT_COUNTER:
        counter = 0
        snapshots.append({
            "gen": gen + 1,
            "fitness": best_genome.fitness,
            "grid": layout_to_grid(sample, best_genome),
        })
best = min(population, key=lambda g: g.fitness if g.fitness is not None else float('inf'))
print("Best fitness:", best.fitness)
if best.scores:
    print("Constraint breakdown:")
    for k, v in best.scores.__dict__.items():
        print(f" - {k}: {v}")


In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
if snapshots:
    ncols = min(5, len(snapshots))
    nrows = math.ceil(len(snapshots) / ncols)
    fig, axes = plt.subplots(nrows, ncols, figsize=(ncols * 3, nrows * 3))
    axes_arr = np.atleast_1d(axes).ravel()
    for ax in axes_arr:
        ax.axis('off')
    vmax = len(sample.rooms) - 1 if sample.rooms else 0
    for idx, snap in enumerate(snapshots):
        ax = axes_arr[idx]
        ax.imshow(snap['grid'], cmap='tab20', vmin=-1, vmax=vmax)
        ax.set_title(f"Gen {snap['gen']}\nfitness={snap['fitness']:.3f}", fontsize=9)
        ax.axis('off')
    plt.tight_layout()
    plt.show()
else:
    print('No snapshots captured (check SNAPSHOT_COUNTER).')


## Floorplan reference
Target layout visualized for comparison.


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img_path = None
for ext in ['png', 'jpg', 'jpeg', 'bmp']:
    candidate = floor_dir / f'floorplan.{ext}'
    if candidate.exists():
        img_path = candidate
        break
plt.figure(figsize=(5,5))
if img_path:
    img = mpimg.imread(img_path)
    plt.imshow(img)
    plt.title(f'Floorplan image: {img_path.name}')
else:
    plt.imshow(sample.target_mask, cmap='Greys')
    plt.title('Fallback: target mask (centroid cells)')
plt.axis('off')
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8,4))
plt.plot(history, marker='o')
plt.title('Best fitness per generation')
plt.xlabel('Generation')
plt.ylabel('Fitness (lower is better)')
plt.grid(True)
plt.tight_layout()
plt.show()
